Process a single recording. This involves:

- Define a configuration
- Read in the time data
- Preprocess the time data
- Decimate the time data
- Window the decimated data
- Calculate the spectra
- Calculate spectral values at evaluation frequencies
- Do the regression

In [89]:
from pathlib import Path
from resistics.config import Configuration
from resistics.time import TimeReaderNumpy
from resistics.gather import QuickGather
from resistics.regression import RegressionInputData, SolverLinear, Solution
from regressioninc.linear import M_estimate

In [90]:
class SolverMestimate(SolverLinear):
    n_jobs: int = -2
    """Number of jobs to run"""

    def run(self, regression_input: RegressionInputData) -> Solution:
        """Run ordinary least squares regression on the RegressionInputData"""
        model = M_estimate()
        return self._solve(regression_input, model)

In [91]:
import numpy as np
from regressioninc.linear import Regressor
import statsmodels.api as sm

class SolverRobust(SolverMestimate):

    def _get_coef(self, model: Regressor, obs: np.ndarray, preds: np.ndarray) -> np.ndarray:
        model = sm.RLM(obs, preds, M=sm.robust.norms.TukeyBiweight())
        result = model.fit()
        return result.params
    
class SolverRecursive(SolverMestimate):

    def _get_coef(self, model: Regressor, obs: np.ndarray, preds: np.ndarray) -> np.ndarray:
        model = sm.RecursiveLS(obs, preds)
        result = model.fit()
        return result.params

In [92]:
config = Configuration(name="testing")
config.solver= SolverRobust()

In [93]:
dir_path = Path("..", "data", "time", "quick", "kap123")
time_data = TimeReaderNumpy().run(dir_path)
ref_time = time_data.metadata.first_time

2023-05-19 12:26:07.220 | INFO     | resistics.time:run:777 - Reading time series data from ../data/time/quick/kap123
2023-05-19 12:26:07.221 | DEBUG    | resistics.time:run:779 - Reading time series metadata from ../data/time/quick/kap123
2023-05-19 12:26:07.223 | DEBUG    | resistics.time:run:787 - Reading samples from 0 to 361511
2023-05-19 12:26:07.225 | INFO     | resistics.time:read_data:1133 - Data successfully read from ../data/time/quick/kap123
2023-05-19 12:26:07.226 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-05-19 12:26:07.226 | DEBUG    | resistics.time:run:790 - Applying scaling to time series data


In [94]:
for processor in config.time_processors:
    time_data = processor.run(time_data)

2023-05-19 12:26:07.280 | INFO     | resistics.time:run:1532 - Removing nan values from channels ['Hx', 'Hy', 'Hz', 'Ex', 'Ey']
2023-05-19 12:26:07.293 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-05-19 12:26:07.294 | INFO     | resistics.time:run:1607 - Removing mean from channels ['Hx', 'Hy', 'Hz', 'Ex', 'Ey']
2023-05-19 12:26:07.302 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32


In [95]:
dec_params = config.dec_setup.run(time_data.metadata.fs)
dec_data = config.decimator.run(dec_params, time_data)

2023-05-19 12:26:07.333 | INFO     | resistics.decimate:run:759 - Decimating level 0 with factor 1
2023-05-19 12:26:07.334 | INFO     | resistics.decimate:run:759 - Decimating level 1 with factor 4
2023-05-19 12:26:07.335 | INFO     | resistics.time:run:2146 - Resampling data from 0.2 Hz to 0.05 Hz
2023-05-19 12:26:07.368 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-05-19 12:26:07.368 | INFO     | resistics.decimate:run:759 - Decimating level 2 with factor 8
2023-05-19 12:26:07.369 | INFO     | resistics.time:run:2146 - Resampling data from 0.05 Hz to 0.00625 Hz
2023-05-19 12:26:07.375 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-05-19 12:26:07.376 | INFO     | resistics.decimate:run:759 - Decimating level 3 with factor 4
2023-05-19 12:26:07.376 | INFO     | resistics.time:run:2146 - Resampling data from 0.00625 Hz to 0.0015625 Hz
2023-05-19 12:26:07.378 | DEBUG    | resistics.time:__init__:479 - Cr

In [96]:
win_params = config.win_setup.run(dec_data.metadata.n_levels, dec_data.metadata.fs)
win_data = config.windower.run(ref_time, win_params, dec_data)

2023-05-19 12:26:07.407 | INFO     | resistics.window:run:1330 - Windowing decimation level 0
2023-05-19 12:26:07.408 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-05-19 12:26:07.412 | INFO     | resistics.window:run:1336 - 3765 windows, size 128, overlap 32
2023-05-19 12:26:07.413 | INFO     | resistics.window:run:1330 - Windowing decimation level 1
2023-05-19 12:26:07.414 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-05-19 12:26:07.416 | INFO     | resistics.window:run:1336 - 941 windows, size 128, overlap 32
2023-05-19 12:26:07.417 | INFO     | resistics.window:run:1330 - Windowing decimation level 2
2023-05-19 12:26:07.418 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-05-19 12:26:07.421 | INFO     | resistics.window:run:1336 - 117 windows, size 128, overlap 32
2023-05-19 12:26:07.422 | INFO     | resistics.window:run:1330 - Windowing 

In [97]:
spec_data = config.fourier.run(win_data)
eval_data = config.evals.run(dec_params, spec_data)

2023-05-19 12:26:07.451 | INFO     | resistics.spectra:run:487 - Performing fourier transforms of windowed decimated data
2023-05-19 12:26:07.452 | INFO     | resistics.spectra:run:489 - Transforming level 0
2023-05-19 12:26:07.453 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-05-19 12:26:07.493 | INFO     | resistics.spectra:run:489 - Transforming level 1
2023-05-19 12:26:07.494 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-05-19 12:26:07.511 | INFO     | resistics.spectra:run:489 - Transforming level 2
2023-05-19 12:26:07.512 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-05-19 12:26:07.518 | INFO     | resistics.spectra:run:489 - Transforming level 3
2023-05-19 12:26:07.519 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-05-19 12:26:07.521 | INFO     | resistics.spectra:run:504 - Fourier transforms completed
2023-05-19 12:26:07.521 | DE

In [98]:
gathered_data = QuickGather().run(dir_path, dec_params, config.tf, eval_data)
reg_data = config.regression_preparer.run(config.tf, gathered_data)

2023-05-19 12:26:07.562 | INFO     | resistics.gather:run:847 - Quick gathering data for regression prepartion
2023-05-19 12:26:07.564 | INFO     | resistics.regression:run:181 - Preparing regression data
2023-05-19 12:26:07.564 | INFO     | resistics.regression:run:182 - Out chans site: kap123
2023-05-19 12:26:07.564 | INFO     | resistics.regression:run:183 - Out chans: ['Ex', 'Ey']
2023-05-19 12:26:07.565 | INFO     | resistics.regression:run:184 - In chans site: kap123
2023-05-19 12:26:07.565 | INFO     | resistics.regression:run:185 - In chans: ['Hx', 'Hy']
2023-05-19 12:26:07.565 | INFO     | resistics.regression:run:186 - Cross chans site: kap123
2023-05-19 12:26:07.566 | INFO     | resistics.regression:run:187 - Cross chans: ['Hx', 'Hy']
2023-05-19 12:26:07.567 | INFO     | resistics.regression:_get_regression_data:212 - Preparing regression data for 20 frequencies
100%|██████████| 20/20 [00:00<00:00, 14020.74it/s]


In [99]:
soln = config.solver.run(reg_data)
fig = soln.tf.plot(
    soln.freqs,
    soln.components,
    to_plot=["ExHy", "EyHx"],
    x_lim=[1, 5],
    res_lim=[0, 4],
    legend="128",
    symbol="circle",
)
fig.update_layout(height=900)
fig.show()

2023-05-19 12:26:07.623 | INFO     | resistics.regression:_solve:595 - Solving for 20 evaluation frequencies
  0%|          | 0/20 [00:00<?, ?it/s]/home/ringo_dingo/.cache/pypoetry/virtualenvs/resistics-b-6cR9vn-py3.10/lib/python3.10/site-packages/numpy/core/_asarray.py:130: ComplexWarning:

Casting complex values to real discards the imaginary part

100%|██████████| 20/20 [00:00<00:00, 33.38it/s]
